# 1、ConversationTokenBufferMemory的使用

举例1：

In [3]:
# 1.导入相关包
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 2.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.定义ConversationTokenBufferMemory对象
memory = ConversationTokenBufferMemory(
    llm=llm,  # 需要计算token的数量, 所以需要传入大模型
    max_token_limit=10  # 设置token上限, 默认值为2000
)

# 添加对话
memory.save_context({"input": "你好吗？"}, {"output": "我很好，谢谢！"})
memory.save_context({"input": "今天天气如何？"}, {"output": "晴天，25度"})

# 查看当前记忆
print(memory.load_memory_variables({}))

{'history': ''}


举例2：

In [6]:
# 1.导入相关包
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 2.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.定义ConversationTokenBufferMemory对象
memory = ConversationTokenBufferMemory(
    llm=llm,  # 需要计算token的数量, 所以需要传入大模型
    max_token_limit=20  # 设置token上限, 默认值为2000
)

# 添加对话
memory.save_context({"input": "你好吗？"}, {"output": "我很好，谢谢！"})
memory.save_context({"input": "今天天气如何？"}, {"output": "晴天，25度"})

# 查看当前记忆
print(memory.load_memory_variables({}))

{'history': 'AI: 晴天，25度'}


# 2、ConversationSummaryMemory的使用


举例1：如果实例化ConversationSummaryMemory前, 没有历史消息, 可以使用构造方法实例化

In [8]:
# 1.导入相关包
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import ChatOpenAI

# 2.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.定义ConversationSummaryMemory对象
memory = ConversationSummaryMemory(llm=llm)

# 4.存储消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话，你能介绍一下你自己吗？"}, {"output": "当然可以了。我是一个无所不能的小智。"})

# 5.读取消息（总结后的）
print(memory.load_memory_variables({}))

{'history': 'The human greets the AI in Chinese with "你好" (hello), and the AI responds with "怎么了" (what\'s wrong?). The human then asks, "你是谁" (who are you?), to which the AI replies, "我是AI助手小智" (I am AI assistant Xiao Zhi). The human then asks for an introduction, and the AI describes itself as "一个无所不能的小智" (a capable assistant).'}


举例2：如果实例化ConversationSummaryMemory前，已经有历史消息，可以调用from_messages()实例化

In [11]:
# 1.导入相关包
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import ChatOpenAI

# 2.定义ChatMessageHistory对象
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.假设原始消息
history = ChatMessageHistory()
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")

# 4.初始化ConversationSummaryMemory实例
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    # 是生成摘要的原材料 保留完整对话供必要时回溯。当新增对话时，LLM需要结合原始历史生成新摘要
    chat_memory=history,
)

print(memory.load_memory_variables({}))
print()

memory.save_context(inputs={"human": "我的名字叫小明"}, outputs={"AI": "很高兴认识你"})
print(memory.load_memory_variables({}))

# 记录了历史交互的信息
print(memory.chat_memory.messages)

{'history': 'The human greets and asks the AI who it is. The AI responds that it is the AI assistant named Xiao Zhi.'}
{'history': 'The human greets the AI and asks who it is. The AI responds that it is the AI assistant named Xiao Zhi. The human introduces themselves as Xiao Ming, and the AI replies that it is pleased to meet them.'}
[HumanMessage(content='你好，你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}), HumanMessage(content='我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]
